# Module imports

In [1]:
from sympy.physics.mechanics import *
from sympy import trigsimp,symbols,cos,sin,lambdify
from math import radians,degrees
import numpy as np
from scipy import linalg, integrate
init_vprinting()

# Symbolic variables

In [2]:
q1,q2 = dynamicsymbols('q1,q2')
#q1d,q2d = dynamicsymbols('q1 q2',1)
u1,u2 = dynamicsymbols('q1,q2',1)
u1d,u2d = dynamicsymbols('q1,q2',2)
t = dynamicsymbols._t

In [3]:
m1,m2,L1,L2,g=symbols('m1,m2,L1,L2,g')
R,S=symbols('R,S')
a1=symbols('a1')

# Reference frames and points

In [4]:
N = ReferenceFrame('N')
O=Point('O')
B1=N.orientnew('B1','Axis',[q1, N.z])
P1 = O.locatenew('P1',L1*B1.x)
G1 = O.locatenew('G1',(L1/2)*B1.x)
B2=B1.orientnew('B2','Axis',[q2, B1.z])
P2 = P1.locatenew('P2',L2*B2.x)
G2 = P1.locatenew('G2',(L2/2)*B2.x)

# Linear and angular velocities

In [5]:
B1.set_ang_vel(N,u1*N.z)
B2.set_ang_vel(N,(u1+u2)*N.z)
O.set_vel(N,0)
P1.v2pt_theory(O,N,B1)
G1.v2pt_theory(O,N,B1)
P2.v2pt_theory(P1,N,B2)
G2.v2pt_theory(P1,N,B2)

# Kinetic energy

In [6]:
Tt=(1/2)*m1*dot(G1.vel(N),G1.vel(N)) +(1/2)*m2*dot(G2.vel(N),G2.vel(N))
Tr=(1/2)*(m1*L1**2/12)*dot(B1.ang_vel_in(N),B1.ang_vel_in(N)) + (1/2)*(m2*L2**2/12)*dot(B2.ang_vel_in(N),B2.ang_vel_in(N))
T=Tt+Tr
#V1=m*g*dot(G1.pos_from(O),N.y)
#V2=m*g*dot(G2.pos_from(O),N.y)
#V=-R*q1+S*q2
V=0

# Constraint equation differentiated wrt time for specified displacement function

In [7]:
cons1=L1*cos(q1)*u1+L2*cos(q1+q2)*(u1+u2)-2*a1*t

# Lagrange's equations

In [8]:
Forces=[(G1,-m1*g*N.y),(G2,-m2*g*N.y)]
#Forces=[(G1,-m*g*N.y),(G2,-m*g*N.y)]
L=T-V
LM = LagrangesMethod(L, [q1,q2], forcelist = Forces, nonhol_coneqs=[cons1], frame=N)
#LM = LagrangesMethod(L, [q1,q2], forcelist = Forces,frame=N)
LM.form_lagranges_equations()

⎡                                                                             
⎢                    2         L₁⋅g⋅m₁⋅cos(q₁)                                
⎢0.333333333333333⋅L₁ ⋅m₁⋅q₁̈ + ─────────────── + L₁⋅g⋅m₂⋅cos(q₁) + 0.04166666
⎢                                     2                                       
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                       0.5⋅L₁⋅L₂⋅m₂⋅(q₁̇ + q₂
⎣                                                                             

                                                                              
          2                    L₂⋅g⋅m₂⋅(-sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂))   
66666667⋅L₂ ⋅m₂⋅(2⋅q₁̈ + 2⋅q₂̈) + ────────────────────────────────────────────
                                                   

# Mass matrix and right hand side

In [9]:
trigsimp(LM.mass_matrix)

⎡                    2            2                                           
⎢0.333333333333333⋅L₁ ⋅m₁ + 1.0⋅L₁ ⋅m₂ + 1.0⋅L₁⋅L₂⋅m₂⋅cos(q₂) + 0.333333333333
⎢                                                                             
⎢                                                                             
⎣                     L₂⋅m₂⋅(0.5⋅L₁⋅cos(q₂) + 0.333333333333333⋅L₂)           

      2                                                                       
333⋅L₂ ⋅m₂  L₂⋅m₂⋅(0.5⋅L₁⋅cos(q₂) + 0.333333333333333⋅L₂)  -L₁⋅cos(q₁) - L₂⋅co
                                                                              
                                          2                                   
                      0.333333333333333⋅L₂ ⋅m₂                   -L₂⋅cos(q₁ + 

          ⎤
s(q₁ + q₂)⎥
          ⎥
          ⎥
q₂)       ⎦

In [10]:
trigsimp(LM.forcing)

⎡                                      L₁⋅g⋅m₁⋅cos(q₁)                     L₂⋅
⎢0.5⋅L₁⋅L₂⋅m₂⋅(2⋅q₁̇ + q₂̇)⋅sin(q₂)⋅q₂̇ - ─────────────── - L₁⋅g⋅m₂⋅cos(q₁) - 
⎢                                             2                               
⎢                                                                             
⎢                                    ⎛             2                 ⎞        
⎣                         -0.5⋅L₂⋅m₂⋅⎝L₁⋅sin(q₂)⋅q₁̇  + g⋅cos(q₁ + q₂)⎠       

g⋅m₂⋅cos(q₁ + q₂)⎤
────────────────────⎥
      2          ⎥
                 ⎥
                 ⎥
                  ⎦

In [11]:
V

# State space equations - augmentated with constraint equations

In [12]:
mm=LM.mass_matrix_full

In [13]:
ff=LM.forcing_full

In [14]:
subs_dict={L1:1,L2:1,m1:36,m2:36,g:10,a1:1}
mmq=mm.subs(subs_dict)
forcingq=ff.subs(subs_dict)

In [15]:
mmq

⎡1  0            0                      0                      0           ⎤
⎢                                                                          ⎥
⎢0  1            0                      0                      0           ⎥
⎢                                                                          ⎥
⎢0  0   36.0⋅cos(q₂) + 60.0    18.0⋅cos(q₂) + 12.0  -cos(q₁ + q₂) - cos(q₁)⎥
⎢                                                                          ⎥
⎢0  0   18.0⋅cos(q₂) + 12.0           12.0               -cos(q₁ + q₂)     ⎥
⎢                                                                          ⎥
⎣0  0  cos(q₁ + q₂) + cos(q₁)     cos(q₁ + q₂)                 0           ⎦

In [16]:
forcingq

⎡                                                   q₁̇                       
⎢                                                                             
⎢                                                   q₂̇                       
⎢                                                                             
⎢18.0⋅(q₁̇ + q₂̇)⋅sin(q₂)⋅q₂̇ + 180⋅sin(q₁)⋅sin(q₂) + 18.0⋅sin(q₂)⋅q₁̇⋅q₂̇ - 1
⎢                                                                             
⎢      -18.0⋅(q₁̇ + q₂̇)⋅sin(q₂)⋅q₁̇ + 180⋅sin(q₁)⋅sin(q₂) + 18.0⋅sin(q₂)⋅q₁̇⋅
⎢                                                                             
⎢                                        2                          2         
⎣                               (q₁̇ + q₂̇) ⋅sin(q₁ + q₂) + sin(q₁)⋅q₁̇  + 2  

                            ⎤
                           ⎥
                            ⎥
                           ⎥
80⋅cos(q₁)⋅cos(q₂) - 540⋅cos(q₁)⎥
                           ⎥
q₂̇ - 180⋅cos(q₁)⋅cos(q₂)   

# Procedure for computing numerical state space matrices

In [17]:
mmf=lambdify([q1,q2],mmq)
forcingf=lambdify([q1,q2,u1,u2],forcingq)

In [18]:
mmf(1,1)

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 79.45088301, 21.72544151, -0.12415547],
       [ 0.        ,  0.        , 21.72544151, 12.        ,  0.41614684],
       [ 0.        ,  0.        ,  0.12415547, -0.41614684,  0.        ]])

In [19]:
forcingf(1,1,0.5,0.5)

array([[   0.5       ],
       [   0.5       ],
       [-205.4969563 ],
       [  71.11981115],
       [   3.11966517]])

# Computation of derivative of state vector for Runge Kutta method

In [20]:
def dydt(stv,t):
    amat=mmf(stv[0],stv[1])
    bvec=forcingf(stv[0],stv[1],stv[2],stv[3])
    xvec=linalg.solve(amat,bvec)
    return xvec.flatten()

# Initial condition and time steps

In [21]:
stv_in=np.array([radians(70),radians(-140),0,0,0],dtype=float)
nval=9; ts = np.linspace(0, 0.8, nval)

# Runge Kutta method

In [22]:
out=integrate.odeint(dydt,stv_in,ts)

In [23]:
out;

In [24]:
qlist=[q1,q2]
plist=[O,G1,P1,G2,P2]
slist=out

# Positions of important points at time steps

In [25]:
xvalt=np.array([],float)
yvalt=np.array([],float)
for ii in range(0,np.shape(slist)[0]):
    xval=np.array([],float)
    yval=np.array([],float)
    for pdum in plist:
        #print(pdum)
        posv=pdum.pos_from(O).subs(subs_dict)
        #print(posv)
        for i in range(0,len(qlist)):
            posv=posv.express(N).subs({qlist[i]:slist[ii,i]})
        posvx=dot(posv,N.x).evalf()
        posvy=dot(posv,N.y).evalf()
        #print(posvx)
        xval=np.append(xval,posvx)
        yval=np.append(yval,posvy)
    xvalt=np.append(xvalt,xval)
    yvalt=np.append(yvalt,yval)
xvalt=np.reshape(xvalt,(len(slist),len(plist)))
yvalt=np.reshape(yvalt,(len(slist),len(plist)))
val=np.block([xvalt,yvalt])

In [26]:
ts
np.reshape(ts,(9,1))
ts

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])

In [27]:
val[:,5];

In [28]:
np.savetxt('pos.txt',val)

# State vector derivatives at time steps

In [29]:
stv_deriv=np.array([],float)
for ii in range(0,np.shape(slist)[0]):
    amat=mmf(slist[ii,0],slist[ii,1])
    bvec=forcingf(slist[ii,0],slist[ii,1],slist[ii,2],slist[ii,3])
    xvec=linalg.solve(amat,bvec)
    stv_deriv=np.append(stv_deriv,xvec)
stv_deriv=np.reshape(stv_deriv,(len(slist),np.shape(slist)[1]))

In [30]:
stv_deriv;

In [31]:
np.savetxt('stv_deriv.txt',stv_deriv)

# Centre of mass accelerations at time steps

In [32]:
P1.a2pt_theory(O,N,B1)
G1.a2pt_theory(O,N,B1)
G2.a2pt_theory(P1,N,B2);

In [33]:
cmlist=[G1,G2]
qlist=[q1,q2]
qdlist=[u1,u2,u1d,u2d]

In [34]:
axvalt=np.array([],float)
ayvalt=np.array([],float)
for ii in range(0,np.shape(slist)[0]):
    axval=np.array([],float)
    ayval=np.array([],float)
    qdict=subs_dict
    for i in range(0,len(qlist)):
        qdict.update({qlist[i]:slist[ii,i]})
    for i in range(0,len(qdlist)):
        qdict.update({qdlist[i]:stv_deriv[ii,i]})        
    for cmdum in cmlist:
        #print(cmdum)
        accv=cmdum.acc(N).express(N).subs(qdict)
        #print(accv)
        accvx=dot(accv,N.x).evalf()
        accvy=dot(accv,N.y).evalf()
        axval=np.append(axval,accvx)
        ayval=np.append(ayval,accvy)
    axvalt=np.append(axvalt,axval)
    ayvalt=np.append(ayvalt,ayval)
axvalt=np.reshape(axvalt,(len(slist),len(cmlist)))
ayvalt=np.reshape(ayvalt,(len(slist),len(cmlist)))
aval=np.block([axvalt,ayvalt])

In [35]:
aval;

In [36]:
np.savetxt('lin_acc.txt',aval)

# Angular velocity of links at time steps

In [37]:
B2.ang_acc_in(N)

In [38]:
bdlist=[B1,B2]
qdlist=[u1,u2,u1d,u2d]

In [39]:
angavalt=np.array([],float)
for ii in range(0,np.shape(slist)[0]):
    angaval=np.array([],float)
    qdict={}
    for i in range(0,len(qdlist)):
        qdict.update({qdlist[i]:stv_deriv[ii,i]})        
    for bddum in bdlist:
        angav=bddum.ang_acc_in(N).subs(qdict)
        angavz=dot(angav,N.z).evalf()
        angaval=np.append(angaval,angavz)
    angavalt=np.append(angavalt,angaval)
angavalt=np.reshape(angavalt,(len(slist),len(bdlist)))

In [42]:
angavalt;

In [41]:
np.savetxt('ang_acc.txt',angavalt)